Multiomeデータセットは非常に疎なので（約98%のセルがゼロ）、疎行列として符号化することで大きな利益を得られます。
このノートブックは、AmbrosMの[this notebook](https://www.kaggle.com/code/ambrosm/msci-multiome-quickstart)に基づいています。このノートブックはMultiomeデータを扱う最初の素晴らしい試みであり、カグラーにとって、疎と密の表現のパフォーマンスを直接対比できることは有益だと思った。

AmbrosMのノートブックとの違いは、主に以下の通りです。
- 疎なCSRフォーマットでデータを表現しているので、全ての学習データをメモリにロードすることができます（密なフォーマットでデータを表現するのに必要な90GB以上のメモリの代わりに、8GB以下のメモリを使用します）。
- 学習データ全体に対してPCA（実際にはTruncatedSVD）を実行（AmbrosMのノートブックでは6000行4000列の部分集合を扱う必要がありました）。
- 16個の成分を保持（AmbrosMのノートブックでは4個）。
- 50000行にRidge回帰を適用（AmbrosMのノートブックでは6000行）。
- より多くのデータを使用しているにもかかわらず、このノートブックは10分強で実行される (AmbrosMのノートブックは1時間以上かかる)

競技会データは、[this notebook](https://www.kaggle.com/code/fabiencrom/multimodal-single-cell-creating-sparse-data/)で生成された[this dataset](https://www.kaggle.com/datasets/fabiencrom/multimodal-single-cell-as-sparse-matrix) にあらかじめスパース行列としてエンコードされています。

このノートブックではマルチオーム予測のみを生成するので、公開時点で最もスコアの良い公開ノートブックであるVuongLamによる[this notebook](https://www.kaggle.com/code/vuonglam/lgbm-baseline-optuna-drop-constant-cite-task)からCITEseq予測を取っています。

In [1]:
import os, gc, pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style
from matplotlib.ticker import MaxNLocator
import os

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import scipy
import scipy.sparse

# The scoring function (from AmbrosM)

この競技は特別な指標を持つ。すべての行について、y_true と y_pred の間のピアソン相関を計算し、これらの相関係数をすべて平均化するのです。

In [2]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    if y_true.shape != y_pred.shape: raise ValueError("Shapes are different.")
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

# Preprocessing and cross-validation

まず、Multiomeのトレーニング用入力データをすべてロードします。1分もかからないはずです。

In [3]:
%%time
train_inputs = scipy.sparse.load_npz("../dataset/train_multi_inputs_values.sparse.npz")

CPU times: total: 23.2 s
Wall time: 23.2 s


In [4]:
DATA_DIR = "../dataset/"
FP_CELL_METADATA = os.path.join(DATA_DIR,"metadata.csv")

In [5]:
metadata_df = pd.read_csv(FP_CELL_METADATA, index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="multiome"]
metadata_df.shape

(161877, 4)

In [6]:
cell_index =np.load("../dataset/train_multi_targets_idxcol.npz",
                   allow_pickle=True)["index"]
meta = metadata_df.reindex(cell_index)

## PCA / TruncatedSVD

疎な行列に PCA を直接適用することはできません。なぜなら，PCA はまずデータを「センタリング」しなければならず，それによって疎密性が失われるからです。そのため，代わりに `TruncatedSVD` を適用します（これは，ほとんど「センタリングしないPCA」です）．ここで、データをもう少し正規化した方がよいかもしれませんが、ここでは単純にしておきます。

In [7]:
%%time
pca = TruncatedSVD(n_components=16, random_state=1)
train_inputs = pca.fit_transform(train_inputs)

CPU times: total: 3min 51s
Wall time: 3min 50s


In [8]:
train_inputs.shape

(105942, 16)

## Random row selection and conversion of the target data to a dense matrix

残念ながら、sklearn の `Ridge` リグレッサーは疎な行列を入力として受け付けますが、ターゲット値として疎な行列を受け付けません。つまり、ターゲットを密な形式に変換する必要があります。密なターゲットデータと疎な入力データの両方をメモリに収めることはできますが、その場合、Ridge回帰処理のメモリが不足します。したがって、今後は学習データから50 000行の部分集合を使用することにします。

In [9]:
np.random.seed(42)
#all_row_indices = np.arange(train_inputs.shape[0])
#np.random.shuffle(all_row_indices)
#selected_rows_indices = all_row_indices[:50000]

In [10]:
#train_inputs = train_inputs[selected_rows_indices]

In [11]:
train_inputs.shape

(105942, 16)

In [12]:
%%time
train_target = scipy.sparse.load_npz("../dataset/train_multi_targets_values.sparse.npz")

CPU times: total: 10.5 s
Wall time: 10.6 s


In [13]:
train_target.shape

(105942, 23418)

In [14]:
#train_target = train_target[selected_rows_indices]
train_target = train_target.todense()
gc.collect()

11

In [15]:
train_target.shape

(105942, 23418)

## KFold Ridge regression

`sklearn` は、行列の代わりに配列を使用するように文句を言います。残念ながら、kaggle で利用可能な古い `scipy` バージョンでは、疎な配列は提供されず、疎な行列のみが提供されます。そこで、この警告を抑制する。

In [16]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

この Kfold リッジ回帰のコードは、ほとんど AmbrosM の [notebook](https://www.kaggle.com/code/ambrosm/msci-multiome-quickstart) から引用しています。なお、 `sklearn` の `Ridge` は疎な行列を透過的に扱える。このブログ記事](https://dziganto.github.io/Sparse-Matrices-For-Efficient-Machine-Learning/) には、 `sklearn` の他のアルゴリズムで疎な行列を受け付けるものがリストアップされています。

In [17]:
meta_new=meta.reset_index(drop=True)

In [19]:
meta_new.day.unique()

array([2, 3, 4, 7], dtype=int64)

In [18]:
%%time
# Cross-validation
from sklearn.model_selection import GroupKFold
#kf = KFold(n_splits=5, shuffle=True, random_state=1)
kf = GroupKFold(n_splits=3)
score_list = []
for fold, (idx_tr, idx_va) in enumerate(kf.split(train_inputs, groups=meta.donor)):
    model = None
    gc.collect()
    tr_day_idx = meta_new.iloc[idx_tr][meta_new.day!=7].index
    va_day_idx=meta_new[meta_new.day==7].index
    X_tr = train_inputs[tr_day_idx] # creates a copy, https://numpy.org/doc/stable/user/basics.copies.html
    y_tr = train_target[tr_day_idx]
    del idx_tr

    model = Ridge(copy_X=False)
    model.fit(X_tr, y_tr)
    del X_tr, y_tr
    gc.collect()

    # We validate the model
    X_va = train_inputs[va_day_idx]
    y_va = train_target[va_day_idx]
    del idx_va
    y_va_pred = model.predict(X_va)
    mse = mean_squared_error(y_va, y_va_pred)
    corrscore = correlation_score(y_va, y_va_pred)
    del X_va, y_va

    print(f"Fold {fold}: mse = {mse:.5f}, corr =  {corrscore:.3f}")
    score_list.append((mse, corrscore))

# Show overall score
result_df = pd.DataFrame(score_list, columns=['mse', 'corrscore'])
print(f"{Fore.GREEN}{Style.BRIGHT}{train_inputs.shape} Average  mse = {result_df.mse.mean():.5f}; corr = {result_df.corrscore.mean():.3f}{Style.RESET_ALL}")

<timed exec>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fold 0: mse = 2.16204, corr =  0.597


<timed exec>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fold 1: mse = 2.15204, corr =  0.599


<timed exec>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fold 2: mse = 2.15337, corr =  0.599
(105942, 16) Average  mse = 2.15582; corr = 0.598
CPU times: total: 1min 7s
Wall time: 38.6 s


# Retraining

In [20]:
# We retrain the model and then delete the training data, which is no longer needed
model, score_list, result_df = None, None, None # free the RAM occupied by the old model
gc.collect()
model = Ridge(copy_X=False) # we overwrite the training data
model.fit(train_inputs, train_target)

Ridge(copy_X=False)

In [21]:
del train_inputs, train_target # free the RAM
_ = gc.collect()

# Predicting

In [22]:
%%time
multi_test_x = scipy.sparse.load_npz("../dataset/test_multi_inputs_values.sparse.npz")
multi_test_x = pca.transform(multi_test_x)
test_pred = model.predict(multi_test_x)
del multi_test_x
gc.collect()

CPU times: total: 30.6 s
Wall time: 19.7 s


71

# Creating submission

入稿時に表示しなければならないセルをロードします。

In [23]:
%%time
# Read the table of rows and columns required for submission
eval_ids = pd.read_parquet("../dataset/evaluation.parquet")

# Convert the string columns to more efficient categorical types
#eval_ids.cell_id = eval_ids.cell_id.apply(lambda s: int(s, base=16))

eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

CPU times: total: 25 s
Wall time: 21.9 s


In [24]:
# Prepare an empty series which will be filled with predictions
submission = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

投稿を行うために必要なので、元のデータフレームの `index` と `columns` をロードします。

In [25]:
%%time
y_columns = np.load("../dataset/train_multi_targets_idxcol.npz",
                   allow_pickle=True)["columns"]

test_index = np.load("../dataset/test_multi_inputs_idxcol.npz",
                    allow_pickle=True)["index"]

CPU times: total: 31.2 ms
Wall time: 29.7 ms


予測された値を投稿ファイルの正しい行に割り当てる。

In [26]:
cell_dict = dict((k,v) for v,k in enumerate(test_index)) 
assert len(cell_dict)  == len(test_index)

gene_dict = dict((k,v) for v,k in enumerate(y_columns))
assert len(gene_dict) == len(y_columns)

In [27]:
eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))

valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

In [28]:
submission.iloc[valid_multi_rows] = test_pred[eval_ids_cell_num[valid_multi_rows].to_numpy(),
eval_ids_gene_num[valid_multi_rows].to_numpy()]

In [29]:
del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_index, y_columns
gc.collect()

66

In [30]:
submission

row_id    cell_id       gene_id        
0         c2150f55becb  CD86                    NaN
1         c2150f55becb  CD274                   NaN
2         c2150f55becb  CD270                   NaN
3         c2150f55becb  CD155                   NaN
4         c2150f55becb  CD112                   NaN
                                             ...   
65744175  2c53aa67933d  ENSG00000134419    6.545989
65744176  2c53aa67933d  ENSG00000186862    0.033699
65744177  2c53aa67933d  ENSG00000170959    0.036630
65744178  2c53aa67933d  ENSG00000107874    1.455560
65744179  2c53aa67933d  ENSG00000166012    4.894366
Name: target, Length: 65744180, dtype: float32

In [31]:
submission.head()

row_id  cell_id       gene_id
0       c2150f55becb  CD86      NaN
1       c2150f55becb  CD274     NaN
2       c2150f55becb  CD270     NaN
3       c2150f55becb  CD155     NaN
4       c2150f55becb  CD112     NaN
Name: target, dtype: float32

In [32]:
submission.reset_index(drop=True, inplace=True)
submission.index.name = 'row_id'
# with open("partial_submission_multi.pickle", 'wb') as f:
#     pickle.dump(submission, f)
# submission

In [33]:
submission.head()

row_id
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: target, dtype: float32

In [34]:
submission.tail()

row_id
65744175    6.545989
65744176    0.033699
65744177    0.036630
65744178    1.455560
65744179    4.894366
Name: target, dtype: float32

In [35]:
submission.to_csv('../multi_sub/ridge2.csv')

In [36]:
submission

row_id
0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
              ...   
65744175    6.545989
65744176    0.033699
65744177    0.036630
65744178    1.455560
65744179    4.894366
Name: target, Length: 65744180, dtype: float32